# Data Cleaning Draft 1

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
import datetime
import matplotlib.dates as mdates

from scipy import stats
from matplotlib.dates import DateFormatter
from IPython.display import display

# Load and Display Raw Data

In [2]:
db_survey = sqlite3.connect(
    "C:\\Users\\Space\\Documents\\py\\Projects\\TuringCollege\\TechMentalHealth\\DataSet\\mental_health.sqlite"
)

In [3]:
%matplotlib inline
pd.options.display.max_rows = 300000
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 500

# Questions

In [4]:
df_questions = pd.read_sql_query("SELECT * FROM Question", db_survey)

In [5]:
df_questions.count().to_frame()
df_questions = df_questions.rename(
    columns={"questiontext": "QuestionText", "questionid": "QuestionID"}
)
df_questions.style.set_properties(**{"text-align": "left"})

,QuestionText,QuestionID
0,What is your age?,1
1,What is your gender?,2
2,What country do you live in?,3
3,"If you live in the United States, which state or territory do you live in?",4
4,Are you self-employed?,5
5,Do you have a family history of mental illness?,6
6,Have you ever sought treatment for a mental health disorder from a mental health professional?,7
7,How many employees does your company or organization have?,8
8,Is your employer primarily a tech company/organization?,9
9,Does your employer provide mental health benefits as part of healthcare coverage?,10


In [6]:
df_questions.drop(
    df_questions.index[
        [
            3,
            7,
            21,
            22,
            23,
            24,
            25,
            26,
            28,
            34,
            35,
            37,
            40,
            41,
            45,
            47,
            48,
            49,
            55,
            56,
            57,
            58,
            59,
            60,
            61,
            62,
            63,
            68,
            70,
            72,
            73,
            74,
            75,
            87,
            89,
            94,
            95,
            96,
            97,
            98,
            101,
            102,
            103,
        ]
    ],
    inplace=True,
)

In [7]:
df_questions.shape[0]

62

# Answers

In [8]:
df_answers = pd.read_sql_query("SELECT * FROM Answer", db_survey)

In [9]:
df_answers.head()

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1,44,2014,2,1
2,32,2014,3,1
3,31,2014,4,1
4,31,2014,5,1


In [10]:
df_answers = df_answers[["SurveyID", "UserID", "QuestionID", "AnswerText"]]

In [11]:
df_answers.head()

,SurveyID,UserID,QuestionID,AnswerText
0,2014,1,1,37
1,2014,2,1,44
2,2014,3,1,32
3,2014,4,1,31
4,2014,5,1,31


# Merged Tables

In [12]:
df_combined = pd.merge(df_questions, df_answers, on="QuestionID", how="inner")

In [13]:
df_combined = df_combined[
    ["SurveyID", "UserID", "QuestionID", "QuestionText", "AnswerText"]
]
df_combined.head()

,SurveyID,UserID,QuestionID,QuestionText,AnswerText
0,2014,1,1,What is your age?,37
1,2014,2,1,What is your age?,44
2,2014,3,1,What is your age?,32
3,2014,4,1,What is your age?,31
4,2014,5,1,What is your age?,31


In [14]:
df_combined.shape[0]

147749

# QuestionID 1: What is your age?

In [15]:
df_age = pd.read_sql_query(
    "SELECT AnswerText FROM Answer where QuestionID = 1", db_survey
)

In [16]:
df_age["AnswerText"] = df_age["AnswerText"].astype(int)

In [17]:
df_age.sort_values("AnswerText", inplace=True)

In [18]:
df_age = df_age[df_age.AnswerText > 18]
df_age = df_age[df_age.AnswerText < 75]

In [19]:
df_value_counts = df_age.AnswerText.value_counts()
df_age_counts = pd.DataFrame(df_value_counts)
df_age_counts = df_value_counts.reset_index()

In [20]:
df_age_counts = df_age_counts.rename(columns={"index": "Age", "AnswerText": "Count"})
df_age_counts = df_age_counts.set_index("Age")
df_age_counts.head()

,Count
Age,
30,250
29,229
32,227
31,223
28,220


# QuestionID 2: What is your gender?

In [154]:
df_gender = pd.read_sql_query(
    "SELECT AnswerText FROM Answer where QuestionID = 2", db_survey
)

In [155]:
df_value_counts = df_gender.AnswerText.value_counts()
df_gender_counts = pd.DataFrame(df_value_counts)
df_gender_counts = df_value_counts.reset_index()

In [156]:
df_gender_counts = df_gender_counts.rename(
    columns={"index": "Gender", "AnswerText": "Count"}
)

In [157]:
df_gender_counts["Gender"] = df_gender_counts["Gender"].replace(
    {"female": "Female", "male": "Male", "MALE": "Male"}
)
df_gender_counts = df_gender_counts.sort_values("Gender")

In [158]:
df_gender_counts["Gender"] = np.where(
    (df_gender_counts["Gender"] == "Male") | (df_gender_counts["Gender"] == "Female"),
    df_gender_counts["Gender"],
    "Other",
)

In [159]:
df_gender_counts.groupby("Gender").sum()

,Count
Gender,
Female,1024
Male,3043
Other,151


# Question ID 3: What country do you live in?

In [170]:
df_country = pd.read_sql_query('SELECT AnswerText FROM Answer where QuestionID = 3', db_survey)

In [171]:
df_country_counts = df_country.AnswerText.value_counts()
df_country_counts = pd.DataFrame(df_country_counts)
df_country_counts = df_country_counts.reset_index()

In [172]:
df_country_counts = df_country_counts.rename(
    columns={"index": "Country", "AnswerText": "Count"}
)

In [173]:
df_country_counts["Country"] = df_country_counts["Country"].replace(
    {
        "-1": "Other",
        "Bahamas, The": "Bahamas",
        "United States of America": 'United States'
    }
)

In [176]:
df_country_counts = df_country_counts.groupby("Country").sum()
df_country_counts = df_country_counts.sort_values("Count", ascending=False)
df_country_counts.head()

,Count
Country,
United States,2604
United Kingdom,482
Canada,199
Germany,136
Netherlands,98


# Question ID 5: Are you self-employed?

In [204]:
df_employed = pd.read_sql_query('SELECT DISTINCT AnswerText FROM Answer where QuestionID = 5', db_survey)
df_employed

,AnswerText
0,-1
1,1
2,0


In [206]:
df_employed['AnswerText'] = df_employed['AnswerText'].str.replace("1", "YES", "-1", "NO")
df_employed

,AnswerText
0,NaN
1,NaN
2,NaN


# Question ID 6: Do you have a family history of mental illness?

In [201]:
df_illness = pd.read_sql_query('SELECT DISTINCT AnswerText FROM Answer where QuestionID = 6', db_survey)
df_illness

,AnswerText
0,No
1,Yes
2,I don't know


In [202]:
df_illness['AnswerText'] = df_illness['AnswerText'].str.replace("I don't know", "Don't Know")
df_illness

,AnswerText
0,No
1,Yes
2,Don't Know
